<a href="https://colab.research.google.com/github/swarathesh/Judging-a-book-by-its-cover-description-an-A.I-approach/blob/master/judging_a_book_by_its_cover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import urllib.request


from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import concatenate
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

Using TensorFlow backend.


In [0]:
!unzip goodreads-best-books-of-2018.zip

In [0]:
df = pd.read_csv('book_data.csv')
df.head()

,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Kristin Hannah,"Alaska, 1974.Unpredictable. Unforgiving. Untam...",NaN,Kindle Edition,NaN,435 pages,4.33,146505,17438,The Great Alone,Fiction|Historical|Historical Fiction|Audioboo...,https://images.gr-assets.com/books/1501852423l...
1,Madeline Miller,"In the house of Helios, god of the sun and mig...",NaN,Hardcover,9.780000e+12,393 pages,4.34,61357,9369,Circe,Fantasy|Fiction|Fantasy|Mythology|Historical|H...,https://images.gr-assets.com/books/1508879575l...
2,Holly Black,Of course I want to be like them. They’re beau...,NaN,Hardcover,9.780000e+12,370 pages,4.18,62897,11948,The Cruel Prince,Fantasy|Young Adult|Fairies|Fae,https://images.gr-assets.com/books/1493047153l...
3,A.J. Finn,Anna Fox lives alone—a recluse in her New York...,NaN,Kindle Edition,NaN,449 pages,3.97,175678,20645,The Woman in the Window,Mystery|Thriller|Fiction|Suspense|Thriller|Mys...,https://images.gr-assets.com/books/1528225499l...
4,Tara Westover,An unforgettable memoir in the tradition of Th...,NaN,Hardcover,9.780000e+12,334 pages,4.47,128337,16424,Educated,Nonfiction|Autobiography|Memoir|Biography,https://images.gr-assets.com/books/1506026635l...


In [0]:
df.columns

Index(['book_authors', 'book_desc', 'book_edition', 'book_format', 'book_isbn',
       'book_pages', 'book_rating', 'book_rating_count', 'book_review_count',
       'book_title', 'genres', 'image_url'],
      dtype='object')

In [0]:
df = df.drop([ 'book_edition', 'book_format', 'book_isbn'],axis=1)

* so our dataset has a minimum of 2.88 rating and max of 5 rating.

In [0]:
# import os 
# os.mkdir('/content/gdrive/My Drive/dataset/BookImages')

In [0]:
df = df.reset_index()



In [0]:
df.shape

(1141, 10)

# CLean Data 

In [0]:
df.book_pages = df.book_pages.str.extract('(^\d*)')

* There are still NAN in book pages fill it with mean 

In [0]:
dx = df.book_pages.dropna().astype(int).mean()


* so the mean is 

In [0]:
dx

327.9000933706816

In [0]:
df.book_pages  = df.book_pages.fillna('327.9')

In [0]:
df.head()

,index,book_authors,book_desc,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,0,Kristin Hannah,"Alaska, 1974.Unpredictable. Unforgiving. Untam...",435,4.33,146505,17438,The Great Alone,Fiction|Historical|Historical Fiction|Audioboo...,https://images.gr-assets.com/books/1501852423l...
1,1,Madeline Miller,"In the house of Helios, god of the sun and mig...",393,4.34,61357,9369,Circe,Fantasy|Fiction|Fantasy|Mythology|Historical|H...,https://images.gr-assets.com/books/1508879575l...
2,2,Holly Black,Of course I want to be like them. They’re beau...,370,4.18,62897,11948,The Cruel Prince,Fantasy|Young Adult|Fairies|Fae,https://images.gr-assets.com/books/1493047153l...
3,3,A.J. Finn,Anna Fox lives alone—a recluse in her New York...,449,3.97,175678,20645,The Woman in the Window,Mystery|Thriller|Fiction|Suspense|Thriller|Mys...,https://images.gr-assets.com/books/1528225499l...
4,4,Tara Westover,An unforgettable memoir in the tradition of Th...,334,4.47,128337,16424,Educated,Nonfiction|Autobiography|Memoir|Biography,https://images.gr-assets.com/books/1506026635l...


In [0]:
df.book_pages.isna().sum()

0

In [0]:
import re

* first genre 

In [0]:
for i in df.index:
  temp = df.genres[df.index == i].astype(str)
  temp = temp.str.split('|')
  temp = str(temp.str[0:1])
  temp = re.match(r"[^[]*\[([^]]*)\]", temp).groups()[0]
  df.genres[df.index == i ] = str(temp)
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


*first author 


In [0]:
for i in df.index:
  temp = df.book_authors[df.index == i].astype(str)
  temp = temp.str.split('|')
  temp = str(temp.str[0:1])
  temp = re.match(r"[^[]*\[([^]]*)\]", temp).groups()[0]
  df.book_authors[df.index == i ] = str(temp)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
df.book_desc = df.book_desc.fillna('empty')

In [0]:
df.isna().sum()

index                0
book_authors         0
book_desc            0
book_pages           0
book_rating          0
book_rating_count    0
book_review_count    0
book_title           0
genres               0
image_url            4
dtype: int64

* cleaned!

# Load Images into an array 

In [0]:
def load_images(df, input_path):
    images = []
    for i in df.index:
        path = input_path + "/" + str(i) + '.jpg'
       # print(path)
        image = cv2.imread(path)
        outputImage = cv2.resize(image, (32, 32))
        images.append(outputImage)
    return np.array(images)

In [0]:
df = df.dropna()
df.shape

(1137, 10)

In [0]:
input_path = './images/images'
images = load_images(df,input_path)

* Normalize

In [0]:
images = images/255

In [0]:
images.shape

(1137, 32, 32, 3)

# train- test split 
* 90% train 
* 10% test 

In [0]:
    n = 100
    allbooks = random.sample(list(df.index), n)
    split_train = np.around(n * 0.90)
    train = df[df['index'].isin(allbooks[0:int(split_train) ])]
    test = df[df['index'].isin(allbooks[int(split_train):n])]


# Judging a book by its cover.
* load image and its associated rating 

In [0]:
train_Y = train.book_rating
test_Y = test.book_rating

train_images = []
for j in train.index:
  
   train_images.append(images[j-1])

test_images = []
for j in test.index:
 
    test_images.append(images[j-1])

train_Y = np.asarray(train_Y) / 5
test_Y = np.asarray(test_Y) / 5
train_x = np.asarray(train_images)
test_x = np.asarray(test_images)


# Custom Cnn Model 

In [0]:
def create_cnn( height, width, depth, filters=(16, 32, 64), regress=False):
        # initialize the input shape and channel dimension, assuming
        # TensorFlow/channels-last ordering

        filters = np.asarray(filters)
        input_shape = (height, width, depth)
        chanDim = -1

        # define the model input
        inputs = Input(shape=input_shape)

        # loop over the number of filters
        for i in range(filters.shape[0]):
            # if this is the first CONV layer then set the input
            # appropriately
            f = filters[i]
            if i == 0:
                x = inputs

            # CONV => RELU => BN => POOL
            x = Conv2D(f, (3, 3), padding="same")(x)
            x = Activation("relu")(x)
            x = BatchNormalization(axis=chanDim)(x)
            x = MaxPooling2D(pool_size=(2, 2))(x)

        # flatten the volume, then FC => RELU => BN => DROPOUT
        x = Flatten()(x)
        x = Dense(16)(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Dropout(0.5)(x)

        # apply another FC layer, this one to match the number of nodes
        # coming out of the MLP
        x = Dense(4)(x)
        x = Activation("relu")(x)

        # check to see if the regression node should be added
        if regress:
            x = Dense(1, activation="linear")(x)

        # construct the CNN
        model = Model(inputs, x)

        # return the CNN
        return model

In [0]:
model = create_cnn(32, 32, 3, regress=True)

In [0]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [0]:
history  = model.fit(train_x,train_Y,validation_data=(test_x,test_Y),epochs=20,batch_size=8)

Train on 90 samples, validate on 10 samples
Epoch 1/20
90/90 [==============================] - 1s 11ms/step - loss: 199.3523 - val_loss: 98.1140
Epoch 2/20
90/90 [==============================] - 0s 2ms/step - loss: 178.8852 - val_loss: 96.9355
Epoch 3/20
90/90 [==============================] - 0s 2ms/step - loss: 160.9740 - val_loss: 95.4147
Epoch 4/20
90/90 [==============================] - 0s 2ms/step - loss: 142.2316 - val_loss: 93.8714
Epoch 5/20
90/90 [==============================] - 0s 3ms/step - loss: 132.5651 - val_loss: 92.0734
Epoch 6/20
90/90 [==============================] - 0s 2ms/step - loss: 119.5358 - val_loss: 90.7653
Epoch 7/20
90/90 [==============================] - 0s 2ms/step - loss: 118.2869 - val_loss: 91.1290
Epoch 8/20
90/90 [==============================] - 0s 2ms/step - loss: 99.7044 - val_loss: 97.5334
Epoch 9/20
90/90 [==============================] - 0s 2ms/step - loss: 90.0603 - val_loss: 103.5919
Epoch 10/20
90/90 [============================

In [0]:
preds_test = model.predict(test_x)
preds_train = model.predict(train_x)

In [0]:
diff = preds_test.flatten() - test_Y
percentDiff = (diff / test_Y) * 100
absPercentDiff = np.abs(percentDiff)
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)


In [0]:
print(model.evaluate(train_x,train_Y))

90/90 [==============================] - 0s 585us/step
85.18101993136936


In [0]:
train.columns

Index(['index', 'book_authors', 'book_desc', 'book_pages', 'book_rating',
       'book_rating_count', 'book_review_count', 'book_title', 'genres',
       'image_url'],
      dtype='object')

# judging Book by it's description 

In [0]:


    # Continous data
cont = ["book_rating_count", "book_review_count","book_pages"]
cs = MinMaxScaler()

trainCont = cs.fit_transform(train[cont])
testCont = cs.transform(test[cont])

    # Categorical data
categ = ["book_authors", "book_title", "genres"]

for j in range(len(categ)):
    bin = LabelBinarizer().fit(df[categ[j]])
    if j == 0:
            trainFull = np.hstack([trainCont, bin.transform(train[categ[j]])])
            testFull = np.hstack([testCont, bin.transform(test[categ[j]])])
    else:
            trainFull = np.hstack([trainFull, bin.transform(train[categ[j]])])
            testFull = np.hstack([testFull, bin.transform(test[categ[j]])])


In [0]:
def create_mlp(dim, regress=False):
        # define our MLP network
        model = Sequential()
        model.add(Dense(8, input_dim=dim, activation="relu"))
        model.add(Dense(4, activation="relu"))

        # check to see if the regression node should be added
        if regress:
            model.add(Dense(1, activation="linear"))

        # return our model
        return model

In [0]:
model_NN = create_mlp(trainFull.shape[1], regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model_NN.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [0]:
model_NN.fit(trainFull,train_Y,validation_data=(testFull,test_Y),epochs=20,batch_size=8)

Train on 90 samples, validate on 10 samples
Epoch 1/20
90/90 [==============================] - 0s 272us/step - loss: 1.7968 - val_loss: 7.4840
Epoch 2/20
90/90 [==============================] - 0s 235us/step - loss: 1.5952 - val_loss: 7.9898
Epoch 3/20
90/90 [==============================] - 0s 226us/step - loss: 1.3106 - val_loss: 7.7100
Epoch 4/20
90/90 [==============================] - 0s 291us/step - loss: 1.2451 - val_loss: 8.1944
Epoch 5/20
90/90 [==============================] - 0s 253us/step - loss: 1.2523 - val_loss: 8.3798
Epoch 6/20
90/90 [==============================] - 0s 263us/step - loss: 1.5803 - val_loss: 7.4995
Epoch 7/20
90/90 [==============================] - 0s 253us/step - loss: 1.2537 - val_loss: 8.1566
Epoch 8/20
90/90 [==============================] - 0s 259us/step - loss: 1.2675 - val_loss: 7.7765
Epoch 9/20
90/90 [==============================] - 0s 257us/step - loss: 1.0045 - val_loss: 7.1978
Epoch 10/20
90/90 [==============================] - 0s 

In [0]:
diff = preds_test.flatten() - test_Y
percentDiff = (diff / test_Y) * 100
absPercentDiff = np.abs(percentDiff)
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
print(model_NN.evaluate(trainFull,train_Y))

90/90 [==============================] - 0s 71us/step
0.8486645566092597


# Judging Book by its cover and description 

In [0]:
mlp = create_mlp(trainFull.shape[1], regress=False)
cnn = create_cnn(32, 32, 3, regress=False)

* SHOW TIME!!!

In [0]:
combinedInput = concatenate([mlp.output, cnn.output])

In [0]:
 x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

In [0]:
model = Model(inputs=[mlp.input, cnn.input], outputs=x)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)


In [0]:
 m = model.fit(
                [trainFull, train_x], train_Y,
                validation_data=([testFull, test_x], test_Y),
                epochs=20, batch_size=8)


Train on 90 samples, validate on 10 samples
Epoch 1/20
90/90 [==============================] - 0s 2ms/step - loss: 1.0574 - val_loss: 9.8679
Epoch 2/20
90/90 [==============================] - 0s 2ms/step - loss: 1.1491 - val_loss: 9.9923
Epoch 3/20
90/90 [==============================] - 0s 2ms/step - loss: 1.2518 - val_loss: 10.2497
Epoch 4/20
90/90 [==============================] - 0s 2ms/step - loss: 1.0112 - val_loss: 10.4258
Epoch 5/20
90/90 [==============================] - 0s 2ms/step - loss: 1.1563 - val_loss: 10.5534
Epoch 6/20
90/90 [==============================] - 0s 2ms/step - loss: 0.9853 - val_loss: 10.7741
Epoch 7/20
90/90 [==============================] - 0s 2ms/step - loss: 1.6409 - val_loss: 10.6676
Epoch 8/20
90/90 [==============================] - 0s 2ms/step - loss: 1.6274 - val_loss: 10.8516
Epoch 9/20
90/90 [==============================] - 0s 2ms/step - loss: 1.2446 - val_loss: 11.2312
Epoch 10/20
90/90 [==============================] - 0s 3ms/step - 

In [0]:
print(model.evaluate([trainFull,train_x],train_Y))

90/90 [==============================] - 0s 628us/step
0.9356112480163574
